In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only")

Using the GPU!


In [3]:
x_train_nhts = np.load("data/x_train_nhts.npy")
x_test_nhts = np.load("data/x_test_nhts.npy")
 
x_train_images = np.load("data/x_train_images.npy")
x_test_images = np.load("data/x_test_images.npy")
  

y_train = np.load("data/y_train.npy")
y_test = np.load("data/y_test.npy")
print("The sample size of training set is: ", x_train_nhts.shape[0])
print("The sample size of testing set is: ", x_test_nhts.shape[0])

The sample size of training set is:  3556
The sample size of testing set is:  889


In [4]:
# bridge numpy to torch
x_train_nhts_torch = torch.as_tensor(x_train_nhts).float() # specify floats for the inputs
x_train_images_torch = torch.as_tensor(x_train_images).float()
x_test_nhts_torch = torch.as_tensor(x_test_nhts).float()
x_test_images_torch = torch.as_tensor(x_test_images).float()
y_train_torch = torch.as_tensor(y_train[:,0])
y_test_torch = torch.as_tensor(y_test[:,0])
n_train = x_train_nhts.shape[0]
n_test = x_test_nhts.shape[0]
# inputs: x_train_nhts, x_train_images, x_test_nhts, x_test_images, y_train, and y_test; 
K = len(np.unique(y_train))
x_dim = x_train_nhts.shape[1]
# 
pd.value_counts(y_train[:,0])/y_train.shape[0]

2    0.339426
1    0.324241
3    0.250562
0    0.085771
dtype: float64

In [5]:
##### Type 1: with only NHTS dataset.
class NN(nn.Module):  # subclass nn.Module
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(x_dim, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, K)
        self.softmax = nn.Softmax(dim=1)  
    def forward(self, x):
        x = self.fc1(x)
        x = x.relu()
        x = self.fc2(x)
        x = x.relu()
        x = self.fc3(x)
        x = self.softmax(x)
        return x

In [6]:
net = NN().float().to(device)
print(type(net))
optim = torch.optim.Adam(net.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
# 
n_epoches = 500 # so many?
batch_size = 200 

<class '__main__.NN'>


In [7]:
# training
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []
for n_epoch in range(n_epoches):
    # create permutation for batch training
    permutation = torch.randperm(x_train_nhts_torch.size()[0])
    for i in range(0, x_train_nhts_torch.size()[0], batch_size):
        # clear gradients first (for each iteration!)!
        optim.zero_grad()
        # forward pass
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = x_train_nhts_torch[indices].to(device), y_train_torch[indices].to(device)
        batch_y_pred_train = net(batch_x).to(device)
        # loss 
        loss = criterion(batch_y_pred_train.squeeze(), batch_y)
        # compute gradients
        loss.backward()
        # one step optim
        optim.step()

    # eval training accuracy
    y_pred_train = net(x_train_nhts_torch.to(device))
    loss_train = criterion(y_pred_train.squeeze(), y_train_torch.to(device))
    train_losses.append(loss_train)
    _, predict_train = torch.max(y_pred_train, axis = 1)
    accuracy_train = (predict_train == y_train_torch.to(device)).sum().item()/n_train  
    train_accuracies.append(accuracy_train)
    # evaluate testing sets step-wise
    net.eval()
    y_pred_test = net(x_test_nhts_torch.to(device))
    loss_test = criterion(y_pred_test.squeeze(), y_test_torch.to(device))
    test_losses.append(loss_test)
    _, predict_test = torch.max(y_pred_test.to(device), axis = 1)
    accuracy_test = (predict_test == y_test_torch.to(device)).sum().item()/n_test
    test_accuracies.append(accuracy_test)
    # print info    
    if n_epoch % 5 == 0:
        print('Epoch {}: train loss: {}; test loss: {}'.format(n_epoch, loss.item(), loss_test.item()))
        print('Epoch {}: train accuracy: {}; test accuracy: {}'.format(n_epoch, accuracy_train, accuracy_test)) 
# Note: about 60% accuracy for both training and testing. (with n_epoches = 500; batch_size = 200)

Epoch 0: train loss: 1.3827788829803467; test loss: 1.38237464427948
Epoch 0: train accuracy: 0.32424071991001124; test accuracy: 0.3160854893138358
Epoch 5: train loss: 1.3760267496109009; test loss: 1.3755552768707275
Epoch 5: train accuracy: 0.32424071991001124; test accuracy: 0.3160854893138358
Epoch 10: train loss: 1.3738867044448853; test loss: 1.3670028448104858
Epoch 10: train accuracy: 0.31833520809898763; test accuracy: 0.296962879640045
Epoch 15: train loss: 1.3540804386138916; test loss: 1.3576515913009644
Epoch 15: train accuracy: 0.33942632170978626; test accuracy: 0.34195725534308213
Epoch 20: train loss: 1.3575122356414795; test loss: 1.3496174812316895
Epoch 20: train accuracy: 0.33942632170978626; test accuracy: 0.34195725534308213
Epoch 25: train loss: 1.326930046081543; test loss: 1.343945026397705
Epoch 25: train accuracy: 0.33942632170978626; test accuracy: 0.34195725534308213
Epoch 30: train loss: 1.3452272415161133; test loss: 1.3401212692260742
Epoch 30: train 

Epoch 270: train loss: 1.2371588945388794; test loss: 1.256300687789917
Epoch 270: train accuracy: 0.48790776152980875; test accuracy: 0.4780652418447694
Epoch 275: train loss: 1.211729645729065; test loss: 1.2558505535125732
Epoch 275: train accuracy: 0.47412823397075365; test accuracy: 0.47244094488188976
Epoch 280: train loss: 1.2623273134231567; test loss: 1.2552651166915894
Epoch 280: train accuracy: 0.47862767154105734; test accuracy: 0.4735658042744657
Epoch 285: train loss: 1.2444339990615845; test loss: 1.2549265623092651
Epoch 285: train accuracy: 0.4746906636670416; test accuracy: 0.47244094488188976
Epoch 290: train loss: 1.235558271408081; test loss: 1.254415512084961
Epoch 290: train accuracy: 0.47750281214848145; test accuracy: 0.4735658042744657
Epoch 295: train loss: 1.232056736946106; test loss: 1.2540253400802612
Epoch 295: train accuracy: 0.4721597300337458; test accuracy: 0.47131608548931386
Epoch 300: train loss: 1.2614485025405884; test loss: 1.2535786628723145
E

In [ ]:
plt.plot(train_losses, label = "train loss")
plt.plot(test_losses, label = "test loss")
plt.legend()



In [ ]:
plt.plot(train_accuracies, label = "train accuracy")
plt.plot(test_accuracies, label = "test accuracy")
plt.legend()

In [ ]:
torch.save(net.state_dict(), "data/nn_ADAM_3fc")